In [76]:
from pynq import Overlay, allocate, Interrupt
import numpy as np
import time
import asyncio

In [77]:
%pwd

'/root/jupyter_notebooks/array_gain_test'

In [78]:
# Cargar el overlay
ol = Overlay("sistema.bit")

In [79]:
help(ol)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay sistema.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    array_gain_0         : pynq.overlay.DefaultIP
    axi_intc_0           : pynq.overlay.DefaultIP
    zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [80]:
# Definir el tamaño: 8192 elementos
size = 8192

# Asignar buffers contiguos para entrada y salida
input_buffer = allocate(shape=(size,), dtype=np.float32)
output_buffer = allocate(shape=(size,), dtype=np.float32)

In [81]:
# Inicializar datos de entrada y la ganancia
input_buffer[:] = np.random.rand(size).astype(np.float32)
gain = np.float32(2.0)

In [82]:
# Obtener la instancia del IP
kr = ol.array_gain_0

# Mostrar el mapa de registros (opcional)
print("Mapa de Registros:")
print(kr.register_map)

Mapa de Registros:
RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input_r_1 = Register(input_r=write-only),
  input_r_2 = Register(input_r=write-only),
  output_r_1 = Register(output_r=write-only),
  output_r_2 = Register(output_r=write-only),
  gain = Register(gain=write-only)
}


In [83]:
# Escribir direcciones físicas y el parámetro
kr.write(kr.register_map.input_r_1.address, input_buffer.physical_address)
kr.write(kr.register_map.output_r_1.address, output_buffer.physical_address)
gain_int = int(np.frombuffer(gain.tobytes(), dtype=np.uint32)[0])
kr.write(kr.register_map.gain.address, gain_int)

In [84]:
kr._interrupts

{'interrupt': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'array_gain_0/interrupt'}}

In [85]:
kr.interrupt

In [86]:
ol.interrupt_pins

{'array_gain_0/interrupt': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'array_gain_0/interrupt'},
 'axi_intc_0/intr': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'axi_intc_0/intr'}}

In [87]:
# Configurar la interrupción.
# Suponiendo que en el diseño la salida de interrupción del IP está conectada al canal 0
irq = Interrupt("axi_intc_0/intr")


# Define an asynchronous coroutine to wait for the interrupt
async def wait_for_interrupt():
    await irq.wait()  # This will block until the interrupt is triggered
    print("Interrupt received! Kernel computation completed.")
    # Optionally, print results or process buffers
    print("First 10 input values :", input_buffer[:10])
    print("First 10 output values:", output_buffer[:10])
    print("Gain used              :", gain)
    irq.clear()  # Clear the interrupt flag


In [ ]:
# Schedule the asynchronous interrupt wait
asyncio.get_event_loop().create_task(wait_for_interrupt())
kr.register_map.CTRL.AP_START = 1
await wait_for_interrupt()

In [59]:
from pynq import PL # se supone que hace clear del PL
PL.reset()

In [60]:
from pynq.pl import PL # aun no he probado si son equivalentes o no.
PL.reset()